In [ ]:
!pip install git+git://github.com/albertbup/deep-belief-network.git
!pip install git+git://github.com/albertbup/deep-belief-network.git@master_gpu

  Cloning git://github.com/albertbup/deep-belief-network.git to /tmp/pip-req-build-2c6fwpcq
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-2c6fwpcq
  Created wheel for deep-belief-network: filename=deep_belief_network-1.0.3-cp36-none-any.whl size=13464 sha256=30408cd6ec2458dce854e2a2de99e040e92d36dfb5c93c092ad69938eb9223c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-m5ep41cn/wheels/29/6d/3b/6a50cf42a32bdfaa903b17832d60d8d3e5dc4b0fd02ae8acaf
Successfully built deep-belief-network
  Cloning git://github.com/albertbup/deep-belief-network.git (to revision master_gpu) to /tmp/pip-req-build-rmu15mjj
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-rmu15mjj
  Running command git checkout -b master_gpu --track origin/master_gpu
  Switched to a new branch 'master_gpu'
  Branch 'master_gpu' set up to track remote branch 'master_gpu' from 'origin'.
     |████████████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from dbn import SupervisedDBNClassification
import pandas as pd
from sklearn.preprocessing import Normalizer
#from sklearn.metrics import confusion_matrix
from scipy.io import arff
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
#load training dataset
dataTrain = arff.loadarff('/content/drive/My Drive/KDDTrain20MultiClass.arff')
traindata = pd.DataFrame(dataTrain[0])
dataTest = arff.loadarff('/content/drive/My Drive/KDDTest21MultiClass.arff')
testdata = pd.DataFrame(dataTrain[0])
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)


In [ ]:
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(traindata, trainlabel)

# Save the model
classifier.save('model.pkl')

# Restore it
classifier = SupervisedDBNClassification.load('model.pkl')

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.038897
>> Epoch 2 finished 	RBM Reconstruction error 0.029303
>> Epoch 3 finished 	RBM Reconstruction error 0.025294
>> Epoch 4 finished 	RBM Reconstruction error 0.024892
>> Epoch 5 finished 	RBM Reconstruction error 0.023549
>> Epoch 6 finished 	RBM Reconstruction error 0.025064
>> Epoch 7 finished 	RBM Reconstruction error 0.024065
>> Epoch 8 finished 	RBM Reconstruction error 0.022727
>> Epoch 9 finished 	RBM Reconstruction error 0.022337
>> Epoch 10 finished 	RBM Reconstruction error 0.021195
>> Epoch 1 finished 	RBM Reconstruction error 0.066889
>> Epoch 2 finished 	RBM Reconstruction error 0.035240
>> Epoch 3 finished 	RBM Reconstruction error 0.024490
>> Epoch 4 finished 	RBM Reconstruction error 0.020956
>> Epoch 5 finished 	RBM Reconstruction error 0.021575
>> Epoch 6 finished 	RBM Reconstruction error 0.020224
>> Epoch 7 finished 	RBM Reconstruction error 0.020617
>> Epoch 8 finished 	RBM Reconstructi

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
expected = testlabel
predicted = classifier.predict(testdata)
conmat = confusion_matrix(expected,predicted)
print(conmat)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))

[[23168     0     0 ...     0     0     0]
 [  374     0     0 ...     0     0     0]
 [  165     0     0 ...     0     0     0]
 ...
 [    1     0     0 ...     0     0     0]
 [    1     0     0 ...     0     0     0]
 [    1     0     0 ...     0     0     0]]
Accuracy_Train: 0.919657033979041
Precision_Train: 0.8457690601471269
Recall_Train: 0.919657033979041
f1-score_Train: 0.8811668388431109


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#test accuracy
import sklearn.metrics as metrics
expected = testlabel
predicted = classifier.predict(testdata)
print("Accuracy:",metrics.accuracy_score(expected, predicted))
print("Precision:",metrics.precision_score(expected, predicted, average='weighted'))
print("Recall:",metrics.recall_score(expected, predicted, average='weighted'))
print("f1-score:",metrics.f1_score(expected, predicted, average='weighted'))


Accuracy: 0.919657033979041
Precision: 0.8457690601471269
Recall: 0.919657033979041
f1-score: 0.8811668388431109


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
